In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Glaciotrack Quickstart\n",
    "\n",
    "This notebook demonstrates how to fetch a glacier velocity dataset from NASA's ITS_LIVE service and compute a simple seasonal displacement time series.\n",
    "\n",
    "📦 Requirements: environment.yml (with `itslive-py`, xarray, matplotlib, pandas)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Imports\n",
    "import matplotlib.pyplot as plt\n",
    "import pandas as pd\n",
    "import xarray as xr\n",
    "from itslive import ITS_LIVE\n",
    "\n",
    "print(\"Libraries imported.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Define Area of Interest (AOI)\n",
    "For demo, we’ll use Columbia Glacier, Alaska (approximate bounding box)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Columbia Glacier bounding box (lon_min, lat_min, lon_max, lat_max)\n",
    "aoi = [-147.2, 60.8, -146.7, 61.2]\n",
    "aoi"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Query ITS_LIVE\n",
    "Use the itslive-py client to fetch a velocity time series cube for the AOI."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize ITS_LIVE client\n",
    "client = ITS_LIVE()\n",
    "\n",
    "# Search velocity mosaics for AOI\n",
    "datasets = client.search(aoi)\n",
    "print(f\"Found {len(datasets)} datasets\")\n",
    "datasets[:3]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Open first dataset (NetCDF via S3)\n",
    "url = datasets[0]['url']\n",
    "ds = xr.open_dataset(url)\n",
    "ds"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Compute simple displacement proxy\n",
    "Here we’ll average velocity magnitude over the AOI and integrate seasonally."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "vel = ds['v']  # velocity magnitude\n",
    "\n",
    "# Convert to pandas series\n",
    "time = pd.to_datetime(ds['time'].values)\n",
    "series = vel.mean(dim=(\"x\", \"y\")).to_pandas()\n",
    "series.index = time\n",
    "\n",
    "# Resample monthly and integrate\n",
    "monthly = series.resample('M').mean()\n",
    "disp = (monthly / 30).cumsum()  # rough displacement in pixels/month\n",
    "disp.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Plot results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "fig, ax = plt.subplots(figsize=(10,4))\n",
    "disp.plot(ax=ax, label=\"Cumulative displacement proxy\")\n",
    "ax.set_ylabel(\"Relative displacement (arb. units)\")\n",
    "ax.set_title(\"Seasonal glacier displacement — Columbia Glacier\")\n",
    "ax.legend()\n",
    "plt.show()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
